# exploration of historical forecast API (open-meteo)
conclusion is that predictions for each predicted hour only have 1 row, so predictions are only 0-2 hours into the future. making this non-fitting for testing our model.

In [ ]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timezone
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define locations
locations = [
    ("Eelde", 53.214469, 6.566481),
    ("DeBilt", 52.108499, 5.180616),
    ("Vlissingen", 51.455677, 3.576488),
    ("Sittard", 50.998299, 5.86291)
]

start_date = "2025-03-05"
end_date = "2025-03-15"


# Open-Meteo API parameters
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
    "latitude": [loc[1] for loc in locations],
    "longitude": [loc[2] for loc in locations],
    "start_date": start_date,
	"end_date": end_date,
    "hourly": ["temperature_2m", "apparent_temperature", "cloud_cover", "wind_speed_10m", "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation", "apparent_temperature"]
}


# Fetch data from Open-Meteo
responses = openmeteo.weather_api(url, params=params)

# Store results in a list
dataframes = []
for i, response in enumerate(responses):
    location_name = locations[i][0]
    
    # Process hourly data
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "apparent_temperature": hourly.Variables(1).ValuesAsNumpy(),
        "cloud_cover": hourly.Variables(2).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(3).ValuesAsNumpy(),
        "diffuse_radiation": hourly.Variables(4).ValuesAsNumpy(),
        "direct_normal_irradiance": hourly.Variables(5).ValuesAsNumpy(),
        "shortwave_radiation": hourly.Variables(6).ValuesAsNumpy(),
        "location": location_name  # Add location column
    }

    # Convert to DataFrame
    df = pd.DataFrame(hourly_data)
    dataframes.append(df)

# Combine all data into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

# Generate timestamp for filename
fetch_timestamp = datetime.now(timezone.utc)
timestamp_str = fetch_timestamp.isoformat(timespec='hours').replace(':', '-')


csv_filename = f"../Data/hist_forecast_weather_{start_date}_to_{end_date}{timestamp_str}.csv"
final_dataframe.to_csv(csv_filename, index=False)


print(f"Data saved to {csv_filename}")

Data saved to ../Data/hist_forecast_weather_2025-03-05_to_2025-03-152025-03-18T12+00-00.csv
